In [10]:
import pandas as pd
import pysam

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing.label import LabelBinarizer

from utils import get_check_sequences_cagi5, save_base_seqs,\
                  get_seqs_and_inds, load_base_seqs


In [3]:
df = pd.read_csv('data/cagi5_df.csv')
df.head()

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element
0,X,138612669,T,A,-0.17,0.07,0,release_F9
1,X,138612669,T,C,-0.26,0.24,-1,release_F9
2,X,138612669,T,G,0.34,0.05,0,release_F9
3,X,138612670,A,C,0.00,0.00,0,release_F9
4,X,138612670,A,G,0.22,0.20,1,release_F9


In [4]:
refs, alts, inds = get_seqs_and_inds(df)

In [5]:
df['ref_sequence'] = refs
df['alt_sequence'] = alts
df['snp_index'] = inds
df['Refcheck'] = df.apply(lambda row: row['ref_sequence'][row['snp_index']], axis=1)
df['Altcheck'] = df.apply(lambda row: row['alt_sequence'][row['snp_index']], axis=1)
import re
df['base_element'] = df.apply(lambda row: row['regulatory_element'][8:], axis=1)
df['base_element'] = df.apply(lambda row: 'TERT' if re.match('TERT', row['base_element']) else row['base_element'], axis=1)

In [49]:
import numpy as np

refp = np.load('data/cagi5_mpra/deepsea_ref_preds.npy')
altp = np.load('data/cagi5_mpra/deepsea_alt_preds.npy')

In [78]:
df['sigdiff'] = df.apply(lambda row: int(row['class'] ==-1 or row['class']==1), axis=1)
df.head()

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element,ref_sequence,alt_sequence,snp_index,Refcheck,Altcheck,base_element,cv_prediction,majority,sigdiff
0,X,138612669,T,A,-0.17,0.07,0,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCA...,45,T,A,F9,1.0,0.0,0
1,X,138612669,T,C,-0.26,0.24,-1,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCC...,45,T,C,F9,1.0,0.0,1
2,X,138612669,T,G,0.34,0.05,0,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCG...,45,T,G,F9,1.0,0.0,0
3,X,138612670,A,C,0.00,0.00,0,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,46,A,C,F9,1.0,0.0,0
4,X,138612670,A,G,0.22,0.20,1,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,46,A,G,F9,1.0,0.0,1


In [112]:
from utils import snp_feats_from_preds

xdiff = snp_feats_from_preds(refp, altp, feattypes=['diff'])

In [113]:
target_col = 'class'

train_df = df[df['base_element']!='TERT']
val_df = df[df['base_element']=='TERT']

X_train = xdiff[train_df.index.values]
y_train = train_df[target_col]

X_val = xdiff[val_df.index.values]
y_val = val_df[target_col]

In [114]:
import xgboost as xgb

xg = xgb.XGBClassifier()

In [119]:
sample_weight = compute_sample_weight('balanced', y_train)
xg.fit(X_train, y_train, 
       eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True)

[0]	validation_0-merror:0.203765	validation_1-merror:0.3375
[1]	validation_0-merror:0.199765	validation_1-merror:0.2975
[2]	validation_0-merror:0.201882	validation_1-merror:0.295
[3]	validation_0-merror:0.200471	validation_1-merror:0.28
[4]	validation_0-merror:0.195529	validation_1-merror:0.2825
[5]	validation_0-merror:0.195765	validation_1-merror:0.265
[6]	validation_0-merror:0.196471	validation_1-merror:0.25
[7]	validation_0-merror:0.195765	validation_1-merror:0.2525
[8]	validation_0-merror:0.196471	validation_1-merror:0.2475
[9]	validation_0-merror:0.195529	validation_1-merror:0.2425
[10]	validation_0-merror:0.196235	validation_1-merror:0.245
[11]	validation_0-merror:0.193647	validation_1-merror:0.23
[12]	validation_0-merror:0.193412	validation_1-merror:0.23
[13]	validation_0-merror:0.192	validation_1-merror:0.21
[14]	validation_0-merror:0.192	validation_1-merror:0.195
[15]	validation_0-merror:0.191059	validation_1-merror:0.195
[16]	validation_0-merror:0.190588	validation_1-merror:0

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [116]:
binarizer = LabelBinarizer()
binarizer.fit_transform(y_val)

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ..., 
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

## TODO
 * to make sure I know what's going on: explicitly set up one vs rest training, validation (by creating is_repressive, is_activating, is_sigdiff classes on df - all of these are of interest as classifications)
 * test non-abs diff features
 * look up reasons for favouring one v rest vs multinomial multiclass classifier

In [120]:
# seems like predicting negs is possible, predicting positives is harder
print(roc_auc_score(ybin, binarizer.transform(xg.predict(X_val)), average=None),
      roc_auc_score(ybin, binarizer.transform(np.zeros(y_val.shape))))

[ 0.49717514  0.49695122  0.5       ] 0.5


In [118]:
binarizer.__dict__

{'classes_': array([-1,  0,  1]),
 'neg_label': 0,
 'pos_label': 1,
 'sparse_input_': False,
 'sparse_output': False,
 'y_type_': 'multiclass'}

In [88]:
print(roc_auc_score(y_val, xg.predict(X_val)), average_precision_score(y_val, xg.predict(X_val)))
print(accuracy_score(y_val, xg.predict(X_val)))
print(accuracy_score(y_val, np.zeros(y_val.shape)))
print(roc_auc_score(y_val, np.zeros(y_val.shape)))

0.619918699187 0.431287878788
0.75
0.82
0.5


In [75]:
np.max(X_val)

0.39666417241096497

In [12]:
from crossval import cvpreds_df
from models import DeepSeaSNP

cvdf = cvpreds_df(df, DeepSeaSNP, model_kwargs={'classifier': 'xgb',
                                                'feattypes':['diff'],
                                                'verbose': False, 
                                                'classifier_kwargs': {'reg_lambda': 10,
                                                                      'booster': 'gbtree'}})

In [11]:
from sklearn.metrics import accuracy_score

binarizer = LabelBinarizer()
ybin = binarizer.fit_transform(cvdf['class'])
print(roc_auc_score(ybin, binarizer.transform(cvdf['cv_prediction']), average=None))

# lr diff: 0.6135, 0.542, 0.6000
# lr absdiff: 0.571, 0.587, 0.517
# gblinear diff: 0.612, 0.57, 0.58
binarizer.classes_

[ 0.61247667  0.57070927  0.58252753]


array([-1,  0,  1])

In [19]:
binarizer = LabelBinarizer()
ybin = binarizer.fit_transform(cvdf[cvdf['base_element']=='TERT']['class'])
print(roc_auc_score(ybin, binarizer.transform(cvdf[cvdf['base_element']=='TERT']['cv_prediction']), average=None))

[ 0.73188406  0.71104336  0.59934183]


In [44]:
cvdf['majority'] = 0.
accuracy_score(cvdf['class'], cvdf['majority'])

0.76129032258064511

In [41]:
cvdf.groupby(['class']).count()

,#Chrom,Pos,Ref,Alt,Value,Confidence,regulatory_element,ref_sequence,alt_sequence,snp_index,Refcheck,Altcheck,base_element,cv_prediction
class,,,,,,,,,,,,,,
-1,736,736,736,736,736,736,736,736,736,736,736,736,736,736
0,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540
1,374,374,374,374,374,374,374,374,374,374,374,374,374,374


In [40]:
cvdf.groupby(['cv_prediction']).count()

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element,ref_sequence,alt_sequence,snp_index,Refcheck,Altcheck,base_element
cv_prediction,,,,,,,,,,,,,,
-1.0,644,644,644,644,644,644,644,644,644,644,644,644,644,644
0.0,3396,3396,3396,3396,3396,3396,3396,3396,3396,3396,3396,3396,3396,3396
1.0,610,610,610,610,610,610,610,610,610,610,610,610,610,610


In [10]:
from utils import snpfeats_from_df, seqfeats_from_df
# absdiff = snpfeats_from_df(df, 1000)
refp, altp = seqfeats_from_df(df, use_gpu=False, seqlen=1000)

<class 'list'>
<class 'list'>


In [12]:
np.save('data/cagi5_mpra/deepsea_ref_preds.npy')

(4650, 919)

In [7]:
df.head()

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element,ref_sequence,alt_sequence,snp_index,Refcheck,Altcheck
0,X,138612669,T,A,-0.17,0.07,0,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCA...,45,T,A
1,X,138612669,T,C,-0.26,0.24,-1,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCC...,45,T,C
2,X,138612669,T,G,0.34,0.05,0,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCG...,45,T,G
3,X,138612670,A,C,0.00,0.00,0,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,46,A,C
4,X,138612670,A,G,0.22,0.20,1,release_F9,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,CCCACTGATGAACTGTGCTGCCACAGTAAATGTAGCCACTATGCCT...,46,A,G


## TODO
 * setup crossval (keep the TERTs together)
 * data exploration: how many of each class are there etc.
 * write a generate feats script that will allow me to run on gpu and save b.c. otherwise it's too slow
   (although now I should be able to use one of my newly trained models on the login node gpu without any trouble)

In [3]:
df[df['Pos'].isin([128413071, 128413072, 128413073, 128413074, 128413075])]

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element
2517,8,128413073,C,A,0.09,0.00,0,release_MYCrs6983267
2518,8,128413073,C,T,-0.03,0.00,0,release_MYCrs6983267
2519,8,128413074,C,A,0.04,0.01,0,release_MYCrs6983267
2520,8,128413074,C,G,-0.01,0.00,0,release_MYCrs6983267
2521,8,128413074,C,T,-0.49,0.13,-1,release_MYCrs6983267
2522,8,128413075,T,A,0.00,0.00,0,release_MYCrs6983267
2523,8,128413075,T,C,-0.06,0.01,0,release_MYCrs6983267
2524,8,128413075,T,G,-0.21,0.04,0,release_MYCrs6983267


In [16]:
with pysam.Fastafile(fasta_file) as genome:
  d0 = genome.fetch('chr8', 128413072, 128413073)
  d1 = genome.fetch('chr8', 128413073, 128413074)
  d2 = genome.fetch('chr8', 128413074, 128413075)
  print(d0, d1, d2)

G C T


In [6]:
df[df['Pos'].isin([396142])]

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element
1034,6,396142,T,A,-0.02,0.00,0,release_IRF4
1035,6,396142,T,C,0.02,0.00,0,release_IRF4
1036,6,396142,T,G,0.15,0.07,0,release_IRF4


In [5]:
with pysam.Fastafile(fasta_file) as genome:
  d0 = genome.fetch('chr6', 396141, 396142)
  d1 = genome.fetch('chr6', 396142, 396143)
  print(d0, d1)

G G


In [3]:
ref, alt, inds = get_sequences_cagi5(df)

G does not match row ref T, position 396142 chr 6 in release_IRF4 CRE
0
G does not match row ref T, position 396142 chr 6 in release_IRF4 CRE
0
G does not match row ref T, position 396142 chr 6 in release_IRF4 CRE
0
G does not match row ref C, position 128413073 chr 8 in release_MYCrs6983267 CRE
0
G does not match row ref C, position 128413073 chr 8 in release_MYCrs6983267 CRE
0
T does not match row ref C, position 37775274 chr 6 in release_ZFAND3 CRE
0
T does not match row ref C, position 37775274 chr 6 in release_ZFAND3 CRE
0


In [7]:
df.head()

,#Chrom,Pos,Ref,Alt,Value,Confidence,class,regulatory_element
0,X,138612669,T,A,-0.17,0.07,0,release_F9
1,X,138612669,T,C,-0.26,0.24,-1,release_F9
2,X,138612669,T,G,0.34,0.05,0,release_F9
3,X,138612670,A,C,0.00,0.00,0,release_F9
4,X,138612670,A,G,0.22,0.20,1,release_F9


In [19]:
df.groupby(['Ref'])['regulatory_element'].max()

Ref
A    release_ZFAND3
C    release_ZFAND3
G    release_ZFAND3
T    release_ZFAND3
Name: regulatory_element, dtype: object

In [15]:
from constants import LOCS

print(len(set(ref)), len(LOCS)) # n.b. this discrepancy is due to the 3 locations where the stated ref sequence does not match the genome

17 14


In [9]:
for i in [0, 123, 457, 999]:
  print(ref[i][inds[i]], alt[i][inds[i]], df.loc[i][['Ref', 'Alt']].values)

T A ['T' 'A']
T A ['T' 'A']
C T ['C' 'T']
G C ['G' 'C']


two options with things that don't match reference:
 * just predict 0
 * just use what they say as the ref

In [59]:
fasta_file = 'data/remote_data/hg19.genome.fa'
f9_start = 138612624
f9_end = 138612924
pos = 138612670
rel_pos = pos-f9_start-1
with pysam.Fastafile(fasta_file) as genome:
  t = genome.fetch('chrX', 138612668, 138612669)
  a = genome.fetch('chrX', 138612669, 138612670)
  print(t,a)
  dnastr = genome.fetch('chrX', f9_start, f9_end)
print(dnastr[rel_pos])

T A
A


In [52]:
seqstarts = df.groupby(['regulatory_element'])['Pos'].min()
seqends = df.groupby(['regulatory_element'])['Pos'].max()
print(seqstarts)
print(seqends)

regulatory_element
release_F9              138612669
release_GP1BB            19710804
release_HBB               5248268
release_HBG1              5271051
release_HNF4A            42984207
release_IRF4               396142
release_IRF6            209989199
release_LDLR             11199906
release_MSMB             51549083
release_MYCrs6983267    128413073
release_PKLR            155271234
release_SORT1           109817337
release_TERT-GBM          1295104
release_TERT-HEK293T      1295104
release_ZFAND3           37775274
Name: Pos, dtype: int64
regulatory_element
release_F9              138612924
release_GP1BB            19711139
release_HBB               5248439
release_HBG1              5271274
release_HNF4A            42984444
release_IRF4               396589
release_IRF6            209989694
release_LDLR             11200177
release_MSMB             51549578
release_MYCrs6983267    128413664
release_PKLR            155271649
release_SORT1           109817736
release_TERT-GBM    

In [46]:
seqlengths = {}
for k, v in seqstarts.iteritems():
  seqlengths[k] = seqends.loc[k] - v
seqlengths

{'release_F9': 255,
 'release_GP1BB': 335,
 'release_HBB': 171,
 'release_HBG1': 223,
 'release_HNF4A': 237,
 'release_IRF4': 447,
 'release_IRF6': 495,
 'release_LDLR': 271,
 'release_MSMB': 495,
 'release_MYCrs6983267': 591,
 'release_PKLR': 415,
 'release_SORT1': 399,
 'release_TERT-GBM': 258,
 'release_TERT-HEK293T': 258,
 'release_ZFAND3': 579}

## TODO 
* look up evaluation metrics for multiclass classification (is there a binary metric like the one below)

a multiple linear regression model of log2(RNA) ~ log2(DNA) + N + offset (were RNA and DNA are counts observed for all n tags, N is a binary matrix associating n tags to m sequence variants and offset is the models' offset normalizing total DNA to RNA counts) was used to assign sequence effects. From the fit, m coefficients (corresponding to the columns of matrix N) were obtained and assigned as the effects of each sequence variant (SNV). A confidence score was derived by capping the p-value of the multiple linear regression at 10-50 and scaling the log10-transformed value between 0 and 1 (i.e. 1 corresponding to a p-value of ≤10-50, 0.5 of 10-25, 0 to a p-value of 1). We deemed a confidence score greater or equal to 0.1 (p-value of 10-5) indicates that the SNV "has an expression effect" while <0.1 means "has no expression effect".

So we want to predict confidence (regression bounded between 0 and 1), a categorical effect given by:
 -1 if value < 0 and confidence > 0.1,
 0  if confidence < 0.1
 1 if value > 0 and confidence > 0.1,
and probability values for the assigned classes (this could come from predict_proba for example)

prediction is categorical, but data isn't ... Direction - Estimated variant effect from the experimental data. Categorical prediction of repressive. (submitted value -1)/activating (submit a value 1)/no effect (submit a value 0), i.e. allele effect is increasing reporter expression (activating), reducing expression (repressive), or not significantly different from zero (confidence score below 0.1).

Two approaches to consider:
(i) directly training a classifier on the class labels
(ii) training to predict value, confidence (regressors), and using these to compute a prediction for the class

You can probably calculate $\Pi_{y = \hat{y}}(p(\hat{y}))\times\Pi_{y\neq\hat{y}}(1-p(\hat{y}))$, which is an approximation to the full categorical cross entropy i suppose

In [13]:
def get_class(row):
  val = row['Value']
  conf = row['Confidence']
  if conf < 0.1:
    return 0
  elif val < 0:
    return -1
  elif val > 0:
    return 1

In [15]:
classes = f9_df.apply(lambda row: get_class(row), axis=1)

In [16]:
classes

0      0
1     -1
2      0
3      0
4      1
5      0
6      0
7      1
8      0
9      0
10     0
11     0
12     0
13     0
14     1
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25    -1
26     0
27     0
28     0
29     0
      ..
206    0
207    0
208    0
209    1
210    0
211    0
212    0
213    1
214    0
215    0
216    1
217    0
218    0
219    1
220    0
221    0
222    0
223    0
224    0
225    0
226    0
227    0
228    0
229    0
230    0
231    0
232    0
233    0
234    0
235    0
Length: 236, dtype: int64